In [ ]:
import random
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras import layers
from utils.graph import draw_learning_curve
from utils.data_loader import create_dataset

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

In [ ]:
SEED = 0
IMG_DIM = 299
BATCH_SIZE = 16
FOLD = 5

In [ ]:
DATA_CSV = f'data/datasplit_fold{FOLD}.csv'

In [ ]:
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
df = pd.read_csv(DATA_CSV)

In [ ]:
train_ds = create_dataset(df=df, batch_size=BATCH_SIZE, mode='train')
val_ds = create_dataset(df=df, batch_size=BATCH_SIZE, mode='val')

In [ ]:
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights='imagenet'
)

In [ ]:
base_model.trainable = False

In [ ]:
inputs = tf.keras.Input(shape=(IMG_DIM, IMG_DIM, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(16, name='logits')(x)
outputs = layers.Activation('softmax', name='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
    loss=tf.keras.losses.CategoricalFocalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
initial_history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
)

In [ ]:
draw_learning_curve(initial_history)

In [ ]:
base_model.trainable = True

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.CategoricalFocalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=20,
    validation_data=val_ds,
)

In [ ]:
draw_learning_curve(history)

In [ ]:
model.save(f'models/fold_{FOLD}.h5')